**데이터과학 기말고사 풀이**

PCA 1번

In [ ]:
import numpy as np
import pandas as pd

file_path = 'gender.csv'

df = pd.read_csv(file_path)

ppd = df[['long_hair', 'forehead_width_cm', 'forehead_height_cm', 'nose_wide', 'nose_long', 'lips_thin', 'distance_nose_to_lip_long']]

print(ppd)

In [1]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

X = ppd
target = df['gender']
pca = PCA(n_components=5)
Y = pca.fit_transform(X)

plt.scatter(Y[:,0], Y[:,1], alpha=0.3)
plt.show()

plt.scatter(Y[:,1], Y[:,2], alpha=0.3)
plt.show()

plt.scatter(Y[:,2], Y[:,3], alpha=0.3)
plt.show()

plt.scatter(Y[:,3], Y[:,4], alpha=0.3)
plt.show()


# plt.plot(pca.explained_variance_, "-o")
# plt.show()


PCA 2번

In [ ]:
print(pca.explained_variance_)

a = [1.35223748, 0.57706972, 0.26194024, 0.11699287, 0.11074894]
b = [round(var, 3) for var in a]
print(b)

PCA 3번

In [ ]:
from numpy import linalg
# covariance matrix, eigen decomposition

X = [[1,2],[2,1],[3,3],[4,5],[5,4]]
X = np.array(X)
X = X - X.mean(axis=0)

# covariance matrix
C = (X.T @ X) / X.shape[0] # 공분산 구하기
L, Q = linalg.eigh(C) # eigen decomposition 수행

print(C)

# print(L)
# print(Q)

# Y = X @ Q

kmeans

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

X = np.array([[1, 1],
              [2, 1],
              [1, 2],
              [4, 3],
              [4, 4]])

init_centers = np.array([[3, 1],
                         [3, 2]])
kmeans = KMeans(n_clusters=2, init=init_centers, n_init=1, max_iter=1)
kmeans.fit(X)

new_centers = kmeans.cluster_centers_
new_centers.sort(axis=0)
new_centers
     

Collaborative Filtering

In [15]:
# user-user, cosine

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Collaborative
v1_mean = np.array([2,3,5,1,5]).mean()
v2_mean = np.array([5,3,1,4]).mean()
v3_mean = np.array([3,4,1,5]).mean()
v4_mean = np.array([2,1,1,5]).mean()
v5_mean = np.array([5,2,2,3]).mean()

v1 = np.array([2,3,5,1,5]) - v1_mean
v2 = np.array([5,3,1,4,v2_mean]) - v2_mean
v3 = np.array([3,v3_mean,4,1,5]) - v3_mean
v4 = np.array([2,1,v4_mean,1,5]) - v4_mean
v5 = np.array([5,2,2,3,v5_mean]) - v5_mean

arrays = np.array([v1, v2, v3, v4, v5])
v3 = v3.reshape(1, -1)
cos = cosine_similarity(arrays, v3)
print(cos)
# res = ((cos[0] * 3) + (cos[3] * 1)) / (cos[0] + cos[3])
# print(res)
# Collaborative + bias
all = np.array([2,3,5,1,5,5,3,1,4,3,4,1,5,2,1,1,5,5,2,2,3]).mean() # 전체 평점 평균
target = np.array([3,4,1,5]).mean() # 평점 매기려는 영화의 평점 평균
user = np.array([5,1,4,1,2]).mean() # 한 유저가 매긴 평점 평균
baseline = all + (target - all) + (user - all) # baseline 값
print(baseline)
res = 2.85 + (((3 - baseline)) + ((1 - baseline))) / 2

print(res)

[[ 0.9212884 ]
 [-0.43571429]
 [ 1.        ]
 [ 0.79264162]
 [-0.17251639]]
2.85
2.0


Latent Factor model

In [ ]:
# Latent Factor Model
# Regularization
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F


items = torch.LongTensor([0,0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3,3,4,4,4])
users = torch.LongTensor([0,1,2,3,4,0,1,3,4,0,1,2,4,0,1,2,3,4,0,2,3])
ratings = torch.FloatTensor([2,5,3,2,5,3,3,1,2,5,1,4,2,1,4,1,1,3,5,5,5])


rank = 1
numItems = items.max() + 1
numUsers = users.max() + 1
P = torch.randn(numItems, rank, requires_grad=True)
Q = torch.randn(numUsers, rank, requires_grad=True)


x = []
y = []

lambda1 = 0.01
lambda2 = 0.01

# 기본 Matrix Factoriztion 구현
optimizer = torch.optim.Adam([P, Q], lr= 0.1)
for epoch in range(1001):
  hypothesis = torch.sum(P[items] * Q[users], dim= 1)
  cost = F.mse_loss(hypothesis ,ratings)

  loss = cost + lambda1 * torch.sum(P ** 2) + lambda2 * torch.sum(Q ** 2)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  x.append(epoch)
  y.append(cost.item())

#   with torch.no_grad():
#     hypo_test = torch.sum(P[items_test] * Q[users_test], dim=1)
#     cost_test = F.mse_loss(hypo_test, ratings_test)
#     y_test.append(cost_test)

  if epoch % 100 == 0:
    print("epoch: {}, cost: {:.6f}" .format(epoch, cost.item()))

# Matplolib으로 epoch마다 Training MSE & Test MSE 그려보기
print(P)
plt.ylabel("MSE")
plt.xlabel("Epoch")
plt.plot(x, y, c="blue", label="Training MSE")
# plt.plot(x,y_test, c="red", label="Test MSE")
plt.legend()
plt.show()